In [1]:
import numpy as np
import pandas as pd
import os



In [2]:

def grab_final_config():
    with open("final_state.xyz", 'r') as f:
        lines = f.readlines()
        return lines

def add_tag(lines, dir):
    energy = float(lines[1].split()[1])
    lines[1] = lines[1].replace('\n','') + ' config: ' + dir + '\n'
    return lines, energy
    
def extract_eam_energy():
    with open("log.lammps", 'r') as f:
        lines = f.readlines()
        min_ene_line = 0
        for i, line in enumerate(lines):
            if '  Energy initial, next-to-last, final =' in line:
                min_ene_line = i + 1
                break
        if not min_ene_line:
            raise Exception("minimization_energy not found")
        eline = lines[min_ene_line]
        eval = float(eline.split()[0])
    return eval

def get_num_atm(lines, atm):
    num_ox = 0
    for line in lines[2:]:
        toks = line.split()
        first_char = toks[0]
        if first_char == atm:
            num_ox += 1
    return num_ox
        
        


In [3]:
os.chdir('../DATA/')
dirs = [d for d in os.listdir('.') if os.path.isdir(d) and d not in  ['.git', '.ipynb_checkpoints' ]]
dirs = sorted(dirs)

dirs


['Al-slab-Olayer1',
 'Al-slab-Olayer2',
 'Al-slab-Olayer4',
 'Al2O3-O',
 'Al2O3-O2',
 'Al2O3-Olayer1',
 'Al2O3-Olayer2',
 'Al2O3-Olayer4',
 'Al2O3-trigonal',
 'AlO-1',
 'AlO-2',
 'Ni-slab-Olayer1',
 'Ni-slab-Olayer2',
 'Ni-slab-Olayer4',
 'NiAlO-1',
 'NiAlO-2',
 'NiO-1',
 'NiO-2',
 'NiO-Slab',
 'NiO-Slab-O',
 'NiO-Slab-O2',
 'O',
 'O2',
 'O2-short',
 'O4-1',
 'O4-2',
 'O4-3',
 'PureAl',
 'PureAl2O3',
 'PureNi',
 'PureNiAl',
 'PureNiO',
 'SlabAl',
 'SlabAlO-1',
 'SlabAlO-2',
 'SlabAlO2-1',
 'SlabAlO2-2',
 'SlabAlO2-s1',
 'SlabAlO2-s10',
 'SlabAlO2-s11',
 'SlabAlO2-s12',
 'SlabAlO2-s13',
 'SlabAlO2-s14',
 'SlabAlO2-s15',
 'SlabAlO2-s16',
 'SlabAlO2-s17',
 'SlabAlO2-s18',
 'SlabAlO2-s19',
 'SlabAlO2-s2',
 'SlabAlO2-s20',
 'SlabAlO2-s21',
 'SlabAlO2-s22',
 'SlabAlO2-s23',
 'SlabAlO2-s24',
 'SlabAlO2-s3',
 'SlabAlO2-s4',
 'SlabAlO2-s5',
 'SlabAlO2-s6',
 'SlabAlO2-s7',
 'SlabAlO2-s8',
 'SlabAlO2-s9',
 'SlabNi',
 'SlabNiO-1',
 'SlabNiO-2',
 'SlabNiO2-1',
 'SlabNiO2-2',
 'SlabNiO2-p1',
 'SlabN

In [4]:

all_configs = []
eam_energies  = []
dft_energies = []
oxygens = []
nickels = []
Als  = []
for dir in dirs:
    os.chdir(dir)
    config_lines = grab_final_config()
    tagged_lines, dft_energy = add_tag(config_lines, dir)
    num_ox = get_num_atm(tagged_lines, "O")
    num_ni = get_num_atm(tagged_lines, "Ni")
    num_al = get_num_atm(tagged_lines, "Al")
    oxygens.append(num_ox)
    nickels.append(num_ni)
    Als.append(num_al)
    dft_energies.append(dft_energy)
    all_configs += tagged_lines
    os.chdir('./EAM_energy')
    try:
        eam_energies.append(extract_eam_energy()) 
    except:
        print(f"dir {dir} is likely an O only dir")
        eam_energies.append(0)
    os.chdir('../../')




dir O is likely an O only dir
dir O2 is likely an O only dir
dir O2-short is likely an O only dir
dir O4-1 is likely an O only dir
dir O4-2 is likely an O only dir
dir O4-3 is likely an O only dir


In [5]:
references = {"Al-slab-Olayer1":"SlabAl",
              "Al-slab-Olayer2":"SlabAl", 
              "Al-slab-Olayer4":"SlabAl",
              "Al2O3-O":"SlabAl",
              "Al2O3-O2":"SlabAl",
              "Al2O3-Olayer1":"SlabAl",
              "Al2O3-Olayer2":"SlabAl",
              "Al2O3-Olayer4":"SlabAl",
              "Al2O3-trigonal":"SlabAl",
              "AlO-1":"PureAl",
              "AlO-2": "PureAl",
              "Ni-slab-Olayer1":"SlabNi",
              "Ni-slab-Olayer2":"SlabNi",
              "Ni-slab-Olayer4":"SlabNi",
              "NiAlO-1":"PureNiAl",
              "NiAlO-2":"PureNiAl",
              "NiO-1": "PureNi",
              "NiO-2": "PureNi",
              "NiO-Slab": "SlabNi",
              "NiO-Slab-O": "SlabNi",
              "NiO-Slab-O2": "SlabNi",
              "O":None,
              "O2":None,
              "O2-short":None,
              "O4-1":None,
              "O4-2":None,
              "O4-3":None,
              "PureAl": None,
              "PureAl2O3": "PureAl",
              "PureNi":None,
              "PureNiAl":None,
              "PureNiO":"PureNi",
              "SlabAl":None,
              "SlabAlO-1":"SlabAl",
              "SlabAlO-2":"SlabAl",
              "SlabAlO2-1":"SlabAl",
              "SlabAlO2-2":"SlabAl",
              "SlabAlO2-s1":"SlabAl",
              "SlabAlO2-s2":"SlabAl",
              "SlabAlO2-s3":"SlabAl",
              "SlabAlO2-s4":"SlabAl",
              "SlabAlO2-s5":"SlabAl",
              "SlabAlO2-s6":"SlabAl",
              "SlabAlO2-s7":"SlabAl",
              "SlabAlO2-s8":"SlabAl",
              "SlabAlO2-s9":"SlabAl",
              "SlabAlO2-s10":"SlabAl",
              "SlabAlO2-s11":"SlabAl",
              "SlabAlO2-s12":"SlabAl",
              "SlabAlO2-s13":"SlabAl",
              "SlabAlO2-s14":"SlabAl",
              "SlabAlO2-s15":"SlabAl",
              "SlabAlO2-s16":"SlabAl",
              "SlabAlO2-s17":"SlabAl",
              "SlabAlO2-s18":"SlabAl",
              "SlabAlO2-s19":"SlabAl",
              "SlabAlO2-s20":"SlabAl",
              "SlabAlO2-s21":"SlabAl",
              "SlabAlO2-s22":"SlabAl",
              "SlabAlO2-s23":"SlabAl",
              "SlabAlO2-s24":"SlabAl",
              "SlabNi": None,
              "SlabNiO-1": "SlabNi",
              "SlabNiO-2": "SlabNi",
              "SlabNiO2-1": "SlabNi",
              "SlabNiO2-2": "SlabNi",
              'SlabNiO2-p1': "SlabNi",
              'SlabNiO2-s01': "SlabNi",
              'SlabNiO2-s02': "SlabNi",
              'SlabNiO2-s03': "SlabNi",
              'SlabNiO2-s04': "SlabNi",
              'SlabNiO2-s05': "SlabNi",
              'SlabNiO2-s06': "SlabNi",
              'SlabNiO2-s07': "SlabNi",
              'SlabNiO2-s08': "SlabNi",
              'SlabNiO2-s09': "SlabNi",
              'SlabNiO2-s10': "SlabNi",
              'SlabNiO2-s11': "SlabNi",
              'SlabNiO2-s12': "SlabNi",
              'SlabNiO2-s13': "SlabNi",
              'SlabNiO2-s14': "SlabNi",
              'SlabNiO2-s15': "SlabNi",
              'SlabNiO2-s16': "SlabNi",
              'SlabNiO2-s17': "SlabNi",
              'SlabNiO2-s18': "SlabNi",
              'SlabNiO2-s19': "SlabNi",
              'SlabNiO2-s20': "SlabNi",
              'SlabNiO2-s21': "SlabNi",
              'SlabNiO2-s22': "SlabNi",
              'SlabNiO2-s23': "SlabNi",
              'SlabNiO2-s24': "SlabNi",
              'SlabNiO2-s25': "SlabNi",
              'SlabNiO2-s26': "SlabNi",
              'SlabNiO2-s27': "SlabNi",
              'SlabNiO2-s28': "SlabNi",
             }

In [6]:
dirs

['Al-slab-Olayer1',
 'Al-slab-Olayer2',
 'Al-slab-Olayer4',
 'Al2O3-O',
 'Al2O3-O2',
 'Al2O3-Olayer1',
 'Al2O3-Olayer2',
 'Al2O3-Olayer4',
 'Al2O3-trigonal',
 'AlO-1',
 'AlO-2',
 'Ni-slab-Olayer1',
 'Ni-slab-Olayer2',
 'Ni-slab-Olayer4',
 'NiAlO-1',
 'NiAlO-2',
 'NiO-1',
 'NiO-2',
 'NiO-Slab',
 'NiO-Slab-O',
 'NiO-Slab-O2',
 'O',
 'O2',
 'O2-short',
 'O4-1',
 'O4-2',
 'O4-3',
 'PureAl',
 'PureAl2O3',
 'PureNi',
 'PureNiAl',
 'PureNiO',
 'SlabAl',
 'SlabAlO-1',
 'SlabAlO-2',
 'SlabAlO2-1',
 'SlabAlO2-2',
 'SlabAlO2-s1',
 'SlabAlO2-s10',
 'SlabAlO2-s11',
 'SlabAlO2-s12',
 'SlabAlO2-s13',
 'SlabAlO2-s14',
 'SlabAlO2-s15',
 'SlabAlO2-s16',
 'SlabAlO2-s17',
 'SlabAlO2-s18',
 'SlabAlO2-s19',
 'SlabAlO2-s2',
 'SlabAlO2-s20',
 'SlabAlO2-s21',
 'SlabAlO2-s22',
 'SlabAlO2-s23',
 'SlabAlO2-s24',
 'SlabAlO2-s3',
 'SlabAlO2-s4',
 'SlabAlO2-s5',
 'SlabAlO2-s6',
 'SlabAlO2-s7',
 'SlabAlO2-s8',
 'SlabAlO2-s9',
 'SlabNi',
 'SlabNiO-1',
 'SlabNiO-2',
 'SlabNiO2-1',
 'SlabNiO2-2',
 'SlabNiO2-p1',
 'SlabN

In [7]:
df = pd.DataFrame({"Species": dirs,  "E_conf_DFT":dft_energies, "E_conf_EAM": eam_energies, "N_O":oxygens, "N_Ni":nickels, "N_Al": Als})
df = df.set_index("Species")
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df)


                 E_conf_DFT  E_conf_EAM  N_O  N_Ni  N_Al
Species                                                 
Al-slab-Olayer1   -1430.764    -800.447   64     0   252
Al-slab-Olayer2   -1559.790    -807.228  128     0   252
Al-slab-Olayer4   -1954.186    -809.148  256     0   252
Al2O3-O           -2019.207    -306.783  163     0   108
Al2O3-O2          -2024.064    -306.938  164     0   108
Al2O3-Olayer1     -2287.489    -306.421  226     0   108
Al2O3-Olayer2     -2307.529    -310.526  290     0   108
Al2O3-Olayer4     -2660.403    -306.445  418     0   108
Al2O3-trigonal    -2059.122    -316.069  162     0   108
AlO-1             -1054.016    -860.023    1     0   256
AlO-2             -1054.201    -859.777    1     0   256
Ni-slab-Olayer1   -1472.690   -1070.544   64   252     0
Ni-slab-Olayer2   -1626.647   -1066.061  128   252     0
Ni-slab-Olayer4   -1812.833   -1052.713  256   252     0
NiAlO-1           -1216.796   -1119.712    1   125   125
NiAlO-2           -1216.627   -

In [8]:
E_ox = np.zeros(len(oxygens)) + df["E_conf_DFT"]["O"]
E_ox

array([0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325,
       0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325,
       0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325,
       0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325,
       0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325,
       0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325,
       0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325,
       0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325,
       0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325,
       0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325,
       0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325,
       0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325,
       0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325, 0.0467325,
       0.0467325, 0.0467325, 0.0467325, 0.0467325, 

In [9]:
df["E_ox"] = E_ox

In [10]:
for i in df["E_conf_DFT"].index.values:
    try:
        print(i, references[i])
    except KeyError:
        print(i, "no_ref")

Al-slab-Olayer1 SlabAl
Al-slab-Olayer2 SlabAl
Al-slab-Olayer4 SlabAl
Al2O3-O SlabAl
Al2O3-O2 SlabAl
Al2O3-Olayer1 SlabAl
Al2O3-Olayer2 SlabAl
Al2O3-Olayer4 SlabAl
Al2O3-trigonal SlabAl
AlO-1 PureAl
AlO-2 PureAl
Ni-slab-Olayer1 SlabNi
Ni-slab-Olayer2 SlabNi
Ni-slab-Olayer4 SlabNi
NiAlO-1 PureNiAl
NiAlO-2 PureNiAl
NiO-1 PureNi
NiO-2 PureNi
NiO-Slab SlabNi
NiO-Slab-O SlabNi
NiO-Slab-O2 SlabNi
O None
O2 None
O2-short None
O4-1 None
O4-2 None
O4-3 None
PureAl None
PureAl2O3 PureAl
PureNi None
PureNiAl None
PureNiO PureNi
SlabAl None
SlabAlO-1 SlabAl
SlabAlO-2 SlabAl
SlabAlO2-1 SlabAl
SlabAlO2-2 SlabAl
SlabAlO2-s1 SlabAl
SlabAlO2-s10 SlabAl
SlabAlO2-s11 SlabAl
SlabAlO2-s12 SlabAl
SlabAlO2-s13 SlabAl
SlabAlO2-s14 SlabAl
SlabAlO2-s15 SlabAl
SlabAlO2-s16 SlabAl
SlabAlO2-s17 SlabAl
SlabAlO2-s18 SlabAl
SlabAlO2-s19 SlabAl
SlabAlO2-s2 SlabAl
SlabAlO2-s20 SlabAl
SlabAlO2-s21 SlabAl
SlabAlO2-s22 SlabAl
SlabAlO2-s23 SlabAl
SlabAlO2-s24 SlabAl
SlabAlO2-s3 SlabAl
SlabAlO2-s4 SlabAl
SlabAlO2-s5 SlabAl
S

In [11]:
erefs_dft = []
erefs_eam = []
n_ref_O = []
n_ref_Ni = []
n_ref_Al = []
ref_structs = []
for struct in dirs:
    print(struct)

    try:
        e_ref_dft = df["E_conf_DFT"][references[struct]]
    except:
        e_ref_dft = np.nan
    # print(e_ref_dft)
    try:
        e_ref_eam = df["E_conf_EAM"][references[struct]]
    except:
        e_ref_eam = np.nan
    # print(e_ref_eam)
    try:
        n_ref_ox = df["N_O"][references[struct]]
    except:
        n_ref_ox = np.nan
    try:
        n_ref_al = df["N_Al"][references[struct]]
    except:
        n_ref_al = np.nan
    try:
        n_ref_ni = df["N_Ni"][references[struct]]
    except:
        n_ref_ni = np.nan
        
    #print(e_ref_eam)
    erefs_dft.append(e_ref_dft)
    erefs_eam.append(e_ref_eam)
    n_ref_O.append(n_ref_ox)
    n_ref_Ni.append(n_ref_ni)
    n_ref_Al.append(n_ref_al)

    
    ref_struct = references[struct]
        
    ref_structs.append(ref_struct)

Al-slab-Olayer1
Al-slab-Olayer2
Al-slab-Olayer4
Al2O3-O
Al2O3-O2
Al2O3-Olayer1
Al2O3-Olayer2
Al2O3-Olayer4
Al2O3-trigonal
AlO-1
AlO-2
Ni-slab-Olayer1
Ni-slab-Olayer2
Ni-slab-Olayer4
NiAlO-1
NiAlO-2
NiO-1
NiO-2
NiO-Slab
NiO-Slab-O
NiO-Slab-O2
O
O2
O2-short
O4-1
O4-2
O4-3
PureAl
PureAl2O3
PureNi
PureNiAl
PureNiO
SlabAl
SlabAlO-1
SlabAlO-2
SlabAlO2-1
SlabAlO2-2
SlabAlO2-s1
SlabAlO2-s10
SlabAlO2-s11
SlabAlO2-s12
SlabAlO2-s13
SlabAlO2-s14
SlabAlO2-s15
SlabAlO2-s16
SlabAlO2-s17
SlabAlO2-s18
SlabAlO2-s19
SlabAlO2-s2
SlabAlO2-s20
SlabAlO2-s21
SlabAlO2-s22
SlabAlO2-s23
SlabAlO2-s24
SlabAlO2-s3
SlabAlO2-s4
SlabAlO2-s5
SlabAlO2-s6
SlabAlO2-s7
SlabAlO2-s8
SlabAlO2-s9
SlabNi
SlabNiO-1
SlabNiO-2
SlabNiO2-1
SlabNiO2-2
SlabNiO2-p1
SlabNiO2-s01
SlabNiO2-s02
SlabNiO2-s03
SlabNiO2-s04
SlabNiO2-s05
SlabNiO2-s06
SlabNiO2-s07
SlabNiO2-s08
SlabNiO2-s11
SlabNiO2-s12
SlabNiO2-s13
SlabNiO2-s14
SlabNiO2-s15
SlabNiO2-s16
SlabNiO2-s17
SlabNiO2-s18
SlabNiO2-s21
SlabNiO2-s22
SlabNiO2-s23
SlabNiO2-s24
SlabNiO2-s25
Sl

In [12]:
df["ref_struct"] = ref_structs
df["E_ref_DFT"] = erefs_dft
df["E_ref_EAM"] = erefs_eam

In [13]:
df

,E_conf_DFT,E_conf_EAM,N_O,N_Ni,N_Al,E_ox,ref_struct,E_ref_DFT,E_ref_EAM
Species,,,,,,,,,
Al-slab-Olayer1,-1430.763575,-800.446608,64,0,252,0.046733,SlabAl,-992.632685,-818.448164
Al-slab-Olayer2,-1559.790087,-807.228302,128,0,252,0.046733,SlabAl,-992.632685,-818.448164
Al-slab-Olayer4,-1954.185789,-809.147611,256,0,252,0.046733,SlabAl,-992.632685,-818.448164
Al2O3-O,-2019.206527,-306.782591,163,0,108,0.046733,SlabAl,-992.632685,-818.448164
Al2O3-O2,-2024.064495,-306.938297,164,0,108,0.046733,SlabAl,-992.632685,-818.448164
...,...,...,...,...,...,...,...,...,...
SlabNiO2-s24,-1421.318767,-1076.802684,2,252,0,0.046733,SlabNi,-1414.467700,-1075.172503
SlabNiO2-s25,-1421.217249,-1076.802684,2,252,0,0.046733,SlabNi,-1414.467700,-1075.172503
SlabNiO2-s26,-1421.043877,-1076.802684,2,252,0,0.046733,SlabNi,-1414.467700,-1075.172503


In [14]:
df["N_O_ref"] = n_ref_O
df["N_Al_ref"] = n_ref_Al
df["N_Ni_ref"] = n_ref_Ni
df["N_Ni_ref"] = n_ref_Ni


In [15]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df)

                 E_conf_DFT  E_conf_EAM  N_O  N_Ni  N_Al   E_ox ref_struct  \
Species                                                                      
Al-slab-Olayer1   -1430.764    -800.447   64     0   252  0.047     SlabAl   
Al-slab-Olayer2   -1559.790    -807.228  128     0   252  0.047     SlabAl   
Al-slab-Olayer4   -1954.186    -809.148  256     0   252  0.047     SlabAl   
Al2O3-O           -2019.207    -306.783  163     0   108  0.047     SlabAl   
Al2O3-O2          -2024.064    -306.938  164     0   108  0.047     SlabAl   
Al2O3-Olayer1     -2287.489    -306.421  226     0   108  0.047     SlabAl   
Al2O3-Olayer2     -2307.529    -310.526  290     0   108  0.047     SlabAl   
Al2O3-Olayer4     -2660.403    -306.445  418     0   108  0.047     SlabAl   
Al2O3-trigonal    -2059.122    -316.069  162     0   108  0.047     SlabAl   
AlO-1             -1054.016    -860.023    1     0   256  0.047     PureAl   
AlO-2             -1054.201    -859.777    1     0   256  0.047 

In [16]:
columns = ['E_conf_DFT', 'E_conf_EAM', 'ref_struct',  'E_ref_DFT', 
       'E_ref_EAM', 'E_ox',  'N_O', 'N_O_ref', 'N_Ni', 'N_Ni_ref', 'N_Al',  'N_Al_ref']

In [17]:
df_resort = df[columns]

In [18]:
df_resort

,E_conf_DFT,E_conf_EAM,ref_struct,E_ref_DFT,E_ref_EAM,E_ox,N_O,N_O_ref,N_Ni,N_Ni_ref,N_Al,N_Al_ref
Species,,,,,,,,,,,,
Al-slab-Olayer1,-1430.763575,-800.446608,SlabAl,-992.632685,-818.448164,0.046733,64,0.0,0,0.0,252,252.0
Al-slab-Olayer2,-1559.790087,-807.228302,SlabAl,-992.632685,-818.448164,0.046733,128,0.0,0,0.0,252,252.0
Al-slab-Olayer4,-1954.185789,-809.147611,SlabAl,-992.632685,-818.448164,0.046733,256,0.0,0,0.0,252,252.0
Al2O3-O,-2019.206527,-306.782591,SlabAl,-992.632685,-818.448164,0.046733,163,0.0,0,0.0,108,252.0
Al2O3-O2,-2024.064495,-306.938297,SlabAl,-992.632685,-818.448164,0.046733,164,0.0,0,0.0,108,252.0
...,...,...,...,...,...,...,...,...,...,...,...,...
SlabNiO2-s24,-1421.318767,-1076.802684,SlabNi,-1414.467700,-1075.172503,0.046733,2,0.0,252,252.0,0,0.0
SlabNiO2-s25,-1421.217249,-1076.802684,SlabNi,-1414.467700,-1075.172503,0.046733,2,0.0,252,252.0,0,0.0
SlabNiO2-s26,-1421.043877,-1076.802684,SlabNi,-1414.467700,-1075.172503,0.046733,2,0.0,252,252.0,0,0.0


In [19]:
df_resort

,E_conf_DFT,E_conf_EAM,ref_struct,E_ref_DFT,E_ref_EAM,E_ox,N_O,N_O_ref,N_Ni,N_Ni_ref,N_Al,N_Al_ref
Species,,,,,,,,,,,,
Al-slab-Olayer1,-1430.763575,-800.446608,SlabAl,-992.632685,-818.448164,0.046733,64,0.0,0,0.0,252,252.0
Al-slab-Olayer2,-1559.790087,-807.228302,SlabAl,-992.632685,-818.448164,0.046733,128,0.0,0,0.0,252,252.0
Al-slab-Olayer4,-1954.185789,-809.147611,SlabAl,-992.632685,-818.448164,0.046733,256,0.0,0,0.0,252,252.0
Al2O3-O,-2019.206527,-306.782591,SlabAl,-992.632685,-818.448164,0.046733,163,0.0,0,0.0,108,252.0
Al2O3-O2,-2024.064495,-306.938297,SlabAl,-992.632685,-818.448164,0.046733,164,0.0,0,0.0,108,252.0
...,...,...,...,...,...,...,...,...,...,...,...,...
SlabNiO2-s24,-1421.318767,-1076.802684,SlabNi,-1414.467700,-1075.172503,0.046733,2,0.0,252,252.0,0,0.0
SlabNiO2-s25,-1421.217249,-1076.802684,SlabNi,-1414.467700,-1075.172503,0.046733,2,0.0,252,252.0,0,0.0
SlabNiO2-s26,-1421.043877,-1076.802684,SlabNi,-1414.467700,-1075.172503,0.046733,2,0.0,252,252.0,0,0.0


In [20]:
df_resort["Al_modifier"] = df_resort["N_Al"] / df_resort["N_Al_ref"]
df_resort["Ni_modifier"] = df_resort["N_Ni"] /df_resort["N_Ni_ref"]

/tmp/ipykernel_6513/2420490869.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["Al_modifier"] = df_resort["N_Al"] / df_resort["N_Al_ref"]
/tmp/ipykernel_6513/2420490869.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["Ni_modifier"] = df_resort["N_Ni"] /df_resort["N_Ni_ref"]


In [21]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df_resort)

                 E_conf_DFT  E_conf_EAM ref_struct  E_ref_DFT  E_ref_EAM  \
Species                                                                    
Al-slab-Olayer1   -1430.764    -800.447     SlabAl   -992.633   -818.448   
Al-slab-Olayer2   -1559.790    -807.228     SlabAl   -992.633   -818.448   
Al-slab-Olayer4   -1954.186    -809.148     SlabAl   -992.633   -818.448   
Al2O3-O           -2019.207    -306.783     SlabAl   -992.633   -818.448   
Al2O3-O2          -2024.064    -306.938     SlabAl   -992.633   -818.448   
Al2O3-Olayer1     -2287.489    -306.421     SlabAl   -992.633   -818.448   
Al2O3-Olayer2     -2307.529    -310.526     SlabAl   -992.633   -818.448   
Al2O3-Olayer4     -2660.403    -306.445     SlabAl   -992.633   -818.448   
Al2O3-trigonal    -2059.122    -316.069     SlabAl   -992.633   -818.448   
AlO-1             -1054.016    -860.023     PureAl  -1046.988   -860.160   
AlO-2             -1054.201    -859.777     PureAl  -1046.988   -860.160   
Ni-slab-Olay

In [22]:
# df_resort["V"] = df_resort["E_conf_DFT"] - df_resort["E_ref_DFT"] - df_resort["E_conf_EAM"] + df_resort["E_ref_EAM"] - (df_resort["N_O"] * df_resort["E_ox"])  

In [23]:
df_resort["E_i_DFT"] = df_resort["E_conf_DFT"] - df_resort["E_ref_DFT"] - df_resort["N_O"]* df_resort["E_ox"]

/tmp/ipykernel_6513/2547235197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["E_i_DFT"] = df_resort["E_conf_DFT"] - df_resort["E_ref_DFT"] - df_resort["N_O"]* df_resort["E_ox"]


In [24]:
df_resort["E_i_EAM"] = df_resort["E_conf_EAM"] - df_resort["E_ref_EAM"]

/tmp/ipykernel_6513/2884512994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["E_i_EAM"] = df_resort["E_conf_EAM"] - df_resort["E_ref_EAM"]


In [25]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df_resort)

                 E_conf_DFT  E_conf_EAM ref_struct  E_ref_DFT  E_ref_EAM  \
Species                                                                    
Al-slab-Olayer1   -1430.764    -800.447     SlabAl   -992.633   -818.448   
Al-slab-Olayer2   -1559.790    -807.228     SlabAl   -992.633   -818.448   
Al-slab-Olayer4   -1954.186    -809.148     SlabAl   -992.633   -818.448   
Al2O3-O           -2019.207    -306.783     SlabAl   -992.633   -818.448   
Al2O3-O2          -2024.064    -306.938     SlabAl   -992.633   -818.448   
Al2O3-Olayer1     -2287.489    -306.421     SlabAl   -992.633   -818.448   
Al2O3-Olayer2     -2307.529    -310.526     SlabAl   -992.633   -818.448   
Al2O3-Olayer4     -2660.403    -306.445     SlabAl   -992.633   -818.448   
Al2O3-trigonal    -2059.122    -316.069     SlabAl   -992.633   -818.448   
AlO-1             -1054.016    -860.023     PureAl  -1046.988   -860.160   
AlO-2             -1054.201    -859.777     PureAl  -1046.988   -860.160   
Ni-slab-Olay

In [26]:
df_resort

,E_conf_DFT,E_conf_EAM,ref_struct,E_ref_DFT,E_ref_EAM,E_ox,N_O,N_O_ref,N_Ni,N_Ni_ref,N_Al,N_Al_ref,Al_modifier,Ni_modifier,E_i_DFT,E_i_EAM
Species,,,,,,,,,,,,,,,,
Al-slab-Olayer1,-1430.763575,-800.446608,SlabAl,-992.632685,-818.448164,0.046733,64,0.0,0,0.0,252,252.0,1.000000,NaN,-441.121769,18.001556
Al-slab-Olayer2,-1559.790087,-807.228302,SlabAl,-992.632685,-818.448164,0.046733,128,0.0,0,0.0,252,252.0,1.000000,NaN,-573.139161,11.219862
Al-slab-Olayer4,-1954.185789,-809.147611,SlabAl,-992.632685,-818.448164,0.046733,256,0.0,0,0.0,252,252.0,1.000000,NaN,-973.516624,9.300553
Al2O3-O,-2019.206527,-306.782591,SlabAl,-992.632685,-818.448164,0.046733,163,0.0,0,0.0,108,252.0,0.428571,NaN,-1034.191239,511.665573
Al2O3-O2,-2024.064495,-306.938297,SlabAl,-992.632685,-818.448164,0.046733,164,0.0,0,0.0,108,252.0,0.428571,NaN,-1039.095939,511.509867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SlabNiO2-s24,-1421.318767,-1076.802684,SlabNi,-1414.467700,-1075.172503,0.046733,2,0.0,252,252.0,0,0.0,NaN,1.0,-6.944532,-1.630181
SlabNiO2-s25,-1421.217249,-1076.802684,SlabNi,-1414.467700,-1075.172503,0.046733,2,0.0,252,252.0,0,0.0,NaN,1.0,-6.843014,-1.630181
SlabNiO2-s26,-1421.043877,-1076.802684,SlabNi,-1414.467700,-1075.172503,0.046733,2,0.0,252,252.0,0,0.0,NaN,1.0,-6.669642,-1.630181


In [27]:
df_resort["Al_modifier"] = df_resort["Al_modifier"].fillna(1.0)
df_resort["Ni_modifier"] = df_resort["Ni_modifier"].fillna(1.0)


/tmp/ipykernel_6513/1492507747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["Al_modifier"] = df_resort["Al_modifier"].fillna(1.0)
/tmp/ipykernel_6513/1492507747.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["Ni_modifier"] = df_resort["Ni_modifier"].fillna(1.0)


In [28]:
df_resort["modifier"]  = df_resort["Al_modifier"]*df_resort["Ni_modifier"]


/tmp/ipykernel_6513/4044517486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["modifier"]  = df_resort["Al_modifier"]*df_resort["Ni_modifier"]


In [29]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df_resort)

                 E_conf_DFT  E_conf_EAM ref_struct  E_ref_DFT  E_ref_EAM  \
Species                                                                    
Al-slab-Olayer1   -1430.764    -800.447     SlabAl   -992.633   -818.448   
Al-slab-Olayer2   -1559.790    -807.228     SlabAl   -992.633   -818.448   
Al-slab-Olayer4   -1954.186    -809.148     SlabAl   -992.633   -818.448   
Al2O3-O           -2019.207    -306.783     SlabAl   -992.633   -818.448   
Al2O3-O2          -2024.064    -306.938     SlabAl   -992.633   -818.448   
Al2O3-Olayer1     -2287.489    -306.421     SlabAl   -992.633   -818.448   
Al2O3-Olayer2     -2307.529    -310.526     SlabAl   -992.633   -818.448   
Al2O3-Olayer4     -2660.403    -306.445     SlabAl   -992.633   -818.448   
Al2O3-trigonal    -2059.122    -316.069     SlabAl   -992.633   -818.448   
AlO-1             -1054.016    -860.023     PureAl  -1046.988   -860.160   
AlO-2             -1054.201    -859.777     PureAl  -1046.988   -860.160   
Ni-slab-Olay

In [30]:
df_resort.columns

Index(['E_conf_DFT', 'E_conf_EAM', 'ref_struct', 'E_ref_DFT', 'E_ref_EAM',
       'E_ox', 'N_O', 'N_O_ref', 'N_Ni', 'N_Ni_ref', 'N_Al', 'N_Al_ref',
       'Al_modifier', 'Ni_modifier', 'E_i_DFT', 'E_i_EAM', 'modifier'],
      dtype='object')

In [31]:
df_resort["V_unscaled"] = df_resort["E_i_DFT"] - df_resort["E_i_EAM"]

/tmp/ipykernel_6513/1473106449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["V_unscaled"] = df_resort["E_i_DFT"] - df_resort["E_i_EAM"]


In [32]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df_resort)

                 E_conf_DFT  E_conf_EAM ref_struct  E_ref_DFT  E_ref_EAM  \
Species                                                                    
Al-slab-Olayer1   -1430.764    -800.447     SlabAl   -992.633   -818.448   
Al-slab-Olayer2   -1559.790    -807.228     SlabAl   -992.633   -818.448   
Al-slab-Olayer4   -1954.186    -809.148     SlabAl   -992.633   -818.448   
Al2O3-O           -2019.207    -306.783     SlabAl   -992.633   -818.448   
Al2O3-O2          -2024.064    -306.938     SlabAl   -992.633   -818.448   
Al2O3-Olayer1     -2287.489    -306.421     SlabAl   -992.633   -818.448   
Al2O3-Olayer2     -2307.529    -310.526     SlabAl   -992.633   -818.448   
Al2O3-Olayer4     -2660.403    -306.445     SlabAl   -992.633   -818.448   
Al2O3-trigonal    -2059.122    -316.069     SlabAl   -992.633   -818.448   
AlO-1             -1054.016    -860.023     PureAl  -1046.988   -860.160   
AlO-2             -1054.201    -859.777     PureAl  -1046.988   -860.160   
Ni-slab-Olay

In [34]:
df_resort["V"] = (df_resort["E_conf_DFT"] - df_resort["modifier"]*df_resort["E_ref_DFT"]) - (df_resort["E_ox"]* df_resort["N_O"])  -(df_resort["E_conf_EAM"] - df_resort["modifier"]*df_resort["E_ref_EAM"]) 

/tmp/ipykernel_6513/1717869976.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["V"] = (df_resort["E_conf_DFT"] - df_resort["modifier"]*df_resort["E_ref_DFT"]) - (df_resort["E_ox"]* df_resort["N_O"])  -(df_resort["E_conf_EAM"] - df_resort["modifier"]*df_resort["E_ref_EAM"])


In [35]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df_resort)

                 E_conf_DFT  E_conf_EAM ref_struct  E_ref_DFT  E_ref_EAM  \
Species                                                                    
Al-slab-Olayer1   -1430.764    -800.447     SlabAl   -992.633   -818.448   
Al-slab-Olayer2   -1559.790    -807.228     SlabAl   -992.633   -818.448   
Al-slab-Olayer4   -1954.186    -809.148     SlabAl   -992.633   -818.448   
Al2O3-O           -2019.207    -306.783     SlabAl   -992.633   -818.448   
Al2O3-O2          -2024.064    -306.938     SlabAl   -992.633   -818.448   
Al2O3-Olayer1     -2287.489    -306.421     SlabAl   -992.633   -818.448   
Al2O3-Olayer2     -2307.529    -310.526     SlabAl   -992.633   -818.448   
Al2O3-Olayer4     -2660.403    -306.445     SlabAl   -992.633   -818.448   
Al2O3-trigonal    -2059.122    -316.069     SlabAl   -992.633   -818.448   
AlO-1             -1054.016    -860.023     PureAl  -1046.988   -860.160   
AlO-2             -1054.201    -859.777     PureAl  -1046.988   -860.160   
Ni-slab-Olay